In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

#scrape
scrape='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
scrapepage= requests.get(scrape).text

#souping tags
soup = BeautifulSoup(scrapepage,'xml')
table = soup.find('table')
Postcode      = []
Borough       = []
Neighbourhood = []

#cleansing
for cell in table.find_all('tr'):
    
    countt=1
    PC=-1
    BC=-1
    NC=-1
    
    #scan and get data
    for dcell in cell.find_all('td'):
        if countt==1: 
            PC=dcell.text
        if countt==2: 
            BC=dcell.text
            tagBorough=dcell.find('a')
            
        if countt==3: 
            NC=str(dcell.text).strip()
            tagNeighbourhood = dcell.find('a')
        countt+=1    
        
        #filter the Not assigned rows and check the neighbours
        if (PC == 'Not assigned' or BC == 'Not assigned' or NC == 'Not assigned'):      
            continue
             
    try:
        if ((tagBorough is None) or (tagNeighbourhood is None)):    
            continue
                 
    except:
        pass
    
    if(PC==-1 or BC==-1 or NC==-1):
        
        continue   
    Postcode.append(PC)
    Borough.append(BC)
    Neighbourhood.append(NC)

up= set(Postcode)
PU= []
BU= []
NU= []

# integrate neighbours
for uniqueelement in up:
    p=''; b=''; n=''; 
    for pid, postcode_element in enumerate(Postcode):
        if uniqueelement == postcode_element:
            p= postcode_element
            b= Borough[pid]
            if n== '': 
                n= Neighbourhood[pid]
            else:
                n= n + ', ' + Neighbourhood[pid]
    PU.append(p)
    BU.append(b)
    NU.append(n)


#to csv

df=pd.DataFrame({'POSTCODE':PU,'BOROUGH':BU,'NEIGHBORHOOD':NU}) 
df.to_csv('File.csv')
df.head(60)


,POSTCODE,BOROUGH,NEIGHBORHOOD
0,M5A,Downtown Toronto,"Harbourfront, Regent Park"
1,M1B,Scarborough,"Rouge, Malvern"
2,M8W,Etobicoke,"Alderwood, Long Branch"
3,M2M,North York,"Newtonbrook, Willowdale"
4,M3L,North York,Downsview West
5,M6S,West Toronto,"Runnymede, Swansea"
6,M4H,East York,Thorncliffe Park
7,M4K,East Toronto,Riverdale
8,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market"
9,M3A,North York,Parkwoods


# SHAPE

In [7]:
df.shape

(77, 3)

# Coordinates

In [8]:
df2=pd.read_csv('http://cocl.us/Geospatial_data')
df2.columns=['POSTCODE', 'LAATITUDE', 'LONGITUDE']

combineddf=pd.merge(df, df2, on=['POSTCODE'], how='inner')

combineddf.head(20)



,POSTCODE,BOROUGH,NEIGHBORHOOD,LAATITUDE,LONGITUDE
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
2,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
3,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
4,M3L,North York,Downsview West,43.739015,-79.506944
5,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
6,M4H,East York,Thorncliffe Park,43.705369,-79.349372
7,M4K,East Toronto,Riverdale,43.679557,-79.352188
8,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049
9,M3A,North York,Parkwoods,43.753259,-79.329656


In [14]:
#only toronto
nh=combineddf[['BOROUGH', 'NEIGHBORHOOD', 'LAATITUDE', 'LONGITUDE']].copy()

torontolist=['Downtown Toronto','West Toronto', 'East Toronto', 'Central Toronto']
nhtoronto=nh[nh.BOROUGH.isin(torontolist)]


# MAP

In [12]:
import folium 
from geopy import Nominatim
address='Toronto, Canada'

l=Nominatim(user_agent="Toronto_explorer")
location=l.geocode(address)
latitude=location.latitude
longitude=location.longitude

#toronto map
torontomap=folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng,borough,neighbourhoods in zip(nhtoronto['LAATITUDE'], nhtoronto['LONGITUDE'], nhtoronto['BOROUGH'], nhtoronto['NEIGHBORHOOD']):
    label='{}, {}'.format(neighbourhoods,borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],radius=5,popup=label,color='green',fill=True,fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(torontomap)  
torontomap